In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
!pip install gcsfs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import gcsfs
import tensorflow as tf
from tensorflow.python.lib.io import file_io

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# path =r'C:/Users/depri/Desktop/OPS813 cloud computing/Project/case1/amex-nyse-nasdaq-stock-histories/fh_20190217/full_history'
fs = gcsfs.GCSFileSystem(project='Case1-TTPRSD')
# allFiles = glob.glob(path + "/*.csv")
allFiles = fs.ls('stockdatagrp21/full_history')
# allFiles
alldf = []
i=0

for file_ in allFiles:
    i+=1
    if i%100 == 0:
      print(i)
#     if i==400:
#         break
    with fs.open(file_) as f:
      dftemp = pd.read_csv(f,index_col=None, header=0)
#     dftemp = pd.read_csv(file_,index_col=None, header=0)
#     dftemp = df_npd.compute()
    #Figure out a way to invert the data frame
    dftemp = dftemp[::-1]
    #Calculating the 200 day moving average before merging all data
    dftemp["200d"] = np.round(dftemp["adjclose"].rolling(window = 200).mean(), 2)
    #Calculating the 50 day moving average before merging all data
    dftemp["50d"] = np.round(dftemp["adjclose"].rolling(window = 50).mean(), 2)
    #Adding a new colmn stock name to identify the stock in question
    dftemp["StockName"]=os.path.basename(file_).split('.')[0]
    alldf.append(dftemp)

df = pd.concat(alldf, axis = 0, ignore_index = True)
df.columns = ["Date", "Volume","Open", "Close", "High", "Low", "AdjustedClose","200d","50d","StockName"]
# print(df) 

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800


In [ ]:
#Finding stocks in the last month with golden cross and visualizing it
allgoldencrossings = []
alldeathcrossings = []
for key, grp in df.groupby('StockName'): # handling data by each stock.
  previous_50 = grp['50d'].shift(1)             # shifting data by 1.
  previous_200 = grp['200d'].shift(1)           # shifting data by 1.
  deathcrossing = ((grp['50d'] <= grp['200d']) & (previous_50 > previous_200))
  goldencrossing = ((grp['50d'] >= grp['200d']) & (previous_50 < previous_200))
  allgoldencrossings.append(goldencrossing)   # Comparing current value of 50d and 200d and comparing previous value of 50d and 200d to see if the golden cross is achieved.
  # figuring out whether I am having golden cross or not.
  alldeathcrossings.append(deathcrossing)
df['GoldenCross'] = pd.concat(allgoldencrossings)
df['DeathCross'] = pd.concat(alldeathcrossings)
df['WeekChange'] = df['AdjustedClose'].pct_change(periods=7)*100
df['TwoWeekChange'] = df['AdjustedClose'].pct_change(periods=14)*100
df['ThreeWeekChange'] = df['AdjustedClose'].pct_change(periods=21)*100
df['MonthChange'] = df['AdjustedClose'].pct_change(periods=30)*100
df['TwoMonthChange'] = df['AdjustedClose'].pct_change(periods=60)*100
df['ThreeMonthChange'] = df['AdjustedClose'].pct_change(periods=90)*100
df['FourMonthChange'] = df['AdjustedClose'].pct_change(periods=120)*100
df['FiveMonthChange'] = df['AdjustedClose'].pct_change(periods=150)*100
df['SixMonthChange'] = df['AdjustedClose'].pct_change(periods=180)*100

# Count of golden crosses
goldenCount = df[df.GoldenCross].GoldenCross.count()

# Count of death crosses
deathCount = df[df.DeathCross].DeathCross.count()
# df
# Percentage of change of 1% in Golden Cross and Death Cross
postiveOnePercentChange = (df[df.GoldenCross & (df.WeekChange > 1)].GoldenCross.count()/goldenCount)*100
negativeOnePercentChange = (df[df.DeathCross & (df.WeekChange < 1)].DeathCross.count()/deathCount)*100
print(postiveOnePercentChange)
print(negativeOnePercentChange)

# Percentage of change of 2% in Golden Cross and Death Cross
postiveTwoPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 2)].GoldenCross.count()/goldenCount)*100
negativeTwoPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 2)].DeathCross.count()/deathCount)*100
print(postiveTwoPercentChange)
print(negativeTwoPercentChange)

# Percentage of change of 3% in Golden Cross and Death Cross
postiveThreePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 3)].GoldenCross.count()/goldenCount)*100
negativeThreePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 3)].DeathCross.count()/deathCount)*100
print(postiveThreePercentChange)
print(negativeThreePercentChange)

# Percentage of change of 4% in Golden Cross and Death Cross
postiveFourPercentChange = (df[df.GoldenCross & (df.MonthChange > 4)].GoldenCross.count()/goldenCount)*100
negativeFourPercentChange = (df[df.DeathCross & (df.MonthChange < 4)].DeathCross.count()/deathCount)*100
print(postiveFourPercentChange)
print(negativeFourPercentChange)

# Percentage of change of 5% in Golden Cross and Death Cross
postiveFivePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 5)].GoldenCross.count()/goldenCount)*100
negativeFivePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 5)].DeathCross.count()/deathCount)*100
print(postiveFivePercentChange)
print(negativeFivePercentChange)

# Percentage of change of 10% in Golden Cross and Death Cross
postiveTenPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 10)].GoldenCross.count()/goldenCount)*100
negativeTenPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 10)].DeathCross.count()/deathCount)*100
print(postiveTenPercentChange)
print(negativeTenPercentChange)

# Percentage of change of 15% in Golden Cross and Death Cross
postiveFifPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 15)].GoldenCross.count()/goldenCount)*100
negativeFifPercentChange = (df[df.DeathCross & (df.FourMonthChange < 15)].DeathCross.count()/deathCount)*100
print(postiveFifPercentChange)
print(negativeFifPercentChange)

# Percentage of change of 20% in Golden Cross and Death Cross
postiveTwenPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 20)].GoldenCross.count()/goldenCount)*100
negativeTwenPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 20)].DeathCross.count()/deathCount)*100
print(postiveTwenPercentChange)
print(negativeTwenPercentChange)

